In [1]:
import param
import panel as pn
import pandas as pd
import time
import requests

from io import StringIO


import plotly.express as px

pn.extension('plotly', 'tabulator', sizing_mode="stretch_width")

In [11]:
class AudioDashboard(param.Parameterized):
    data = param.Dict()
  
    file_input = param.Parameter()
    
    def __init__(self, **params):
        super().__init__(file_input=pn.widgets.TextInput(), **params)
        self.transcript = pn.pane.Str(max_width = 600, height_policy = "fit")
        self.download = pn.widgets.FileDownload(name="Download transcript", filename="transcript.txt", callback=self._download_callback, button_type="primary")

    @pn.depends("file_input.value", watch=True)
    def _parse_file_input(self):
        print("parse input ")
        print(self.file_input)
        value = self.file_input.value
        if value:
            print("in parse")
            self.data = self.get_and_analyze_transcript()
            print(self.data)
            
        else:
            print("error")

    @pn.depends('data', watch=True)
    def get_transcript(self):
        print("get transcript ")
        self.transcript.object = self.data["text"]
        
        
    def _download_callback(self):
        print("download")
        if self.data is not None:
            buffer = StringIO()
            buffer.write(self.data["text"])
            buffer.seek(0)
            print("return buffer")

            return buffer  
        else:
            return
        
        
    def poll(self):
        polling_endpoint = transcript_endpoint + '/' + self.transcript_id
        polling_response = requests.get(polling_endpoint, headers=self.headers)
        self.data = polling_response.json()
        
    def get_and_analyze_transcript(self):
        audio_url = self.file_input.value
                
        if audio_url:
            print("yessss")
            API_KEY = "XXXX"

            endpoint = "https://api.assemblyai.com/v2/transcript"
            audio_url_json = {
                "audio_url": audio_url,
                "sentiment_analysis": True,
                "auto_highlights": True,
                "iab_categories": True,
                "auto_chapters": True
            }
            headers = {
                "authorization": API_KEY,
                "content-type": "application/json"
            }
            transcript_response = requests.post(endpoint, json=audio_url_json, headers=headers)

            result_endpoint = endpoint + '/' + transcript_response.json()["id"]
            headers_auth = {
                "authorization": API_KEY,
            }
            response = requests.get(result_endpoint, headers=headers_auth)
            while response.json()['status'] != "completed":
                response = requests.get(result_endpoint, headers=headers_auth)
                time.sleep(3)
                
            return response.json()

        else:
            print("no")
            return
        
    def view(self):
        return pn.Column(
            "## Transcript",
            self.transcript,
        )
    
audio_app = AudioDashboard()

audio_app_view = audio_app.view()
audio_app_view

Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] Str(None, height_policy='fit', max_width=600, sizing_mode='stretch_width')

In [12]:

description = """
This is my app :)
"""

component = pn.Column(
    description,
    audio_app_view,
    sizing_mode='stretch_both'
)
component

template = pn.template.FastListTemplate(
    title='Audio Content Explorer', 
    sidebar=[pn.pane.Markdown("### Input a link:"), 
             audio_app.file_input, 
             audio_app.download],
    main=[component],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)

template.servable()

Launching server at http://localhost:56641


ERROR:tornado.application:Uncaught exception GET /ws (::1)
HTTPServerRequest(protocol='http', host='localhost:56641', method='GET', uri='/ws', version='HTTP/1.1', remote_ip='::1')
Traceback (most recent call last):
  File "/Users/thuvu/Documents/projects/audio_analyzer_assemblyai/lib/python3.9/site-packages/tornado/websocket.py", line 942, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
  File "/Users/thuvu/Documents/projects/audio_analyzer_assemblyai/lib/python3.9/site-packages/tornado/web.py", line 3208, in wrapper
    return method(self, *args, **kwargs)
  File "/Users/thuvu/Documents/projects/audio_analyzer_assemblyai/lib/python3.9/site-packages/bokeh/server/views/ws.py", line 149, in open
    raise ProtocolError("Token is expired.")
bokeh.protocol.exceptions.ProtocolError: Token is expired.
